In [1]:
!pip install "transformers[torch]" "huggingface_hub[inference]"
!pip install llama-index
!pip install llama-index-llms-huggingface
!pip install llama-index-embeddings-huggingface
!pip install llama-index-embeddings-instructor
!pip install llama-index-llms-ollama
!pip install einops
!pip install autoawq
!pip install llama-index-embeddings-nomic

In [2]:
 #!curl -fsSL https://ollama.com/install.sh | sh

In [1]:
import os
from typing import List, Optional

from llama_index.llms.huggingface import (
    HuggingFaceInferenceAPI,
    HuggingFaceLLM,
)

In [2]:
from llama_index.core import set_global_tokenizer
from transformers import AutoTokenizer

In [3]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding




# Smaller embedding model
#embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5", trust_remote_code = True, device = "cuda")

# Nomic with custom dimensionality

from llama_index.embeddings.nomic import NomicEmbedding

import nest_asyncio

nest_asyncio.apply()

embed_model = NomicEmbedding(
    api_key="nk-hjIl5ZNK1LEC3IP1swmC4sd3jC5LUw5asbbcchyFGGg",
    dimensionality=768,
    model_name="nomic-embed-text-v1.5",
)

In [4]:
embeddings = embed_model.get_text_embedding("Hello World!")
print(len(embeddings))
print(embeddings[:5])

768
[-0.03062439, -0.011756897, -0.1640625, 0.012763977, 0.012458801]


In [5]:
# RUN IN TWO SEPARATE TERMINALS
#!ollama serve
#!ollama run llama3

In [6]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.core.embeddings import resolve_embed_model

from llama_index.llms.ollama import Ollama

Settings.llm = Ollama(model="mistral", request_timeout=30.0)
Settings.embed_model = embed_model

In [7]:
documents = SimpleDirectoryReader("./data").load_data()

In [8]:
index = VectorStoreIndex.from_documents(documents)

In [9]:
query_engine = index.as_query_engine(similarity_top_k=10)

In [10]:
response = query_engine.query("what are the participants of the PATAFEST european project?")
print(response)

 The PATAFEST European project involves a consortium of 18 partners, encompassing entities from the EU and associated countries, such as Israel and Ecuador. These partners represent research and technology organizations (RTOs), a coordinator, agricultural cooperatives, large companies, small and medium-sized enterprises (SMEs), research organizations, universities, and associations. Notable participants include FUNDITEC, NEIKER, UDAPA, ANECOOP, EUPAT, and ECSA.


In [11]:
response = query_engine.query("what are the work packages of the PATAFEST european project?")
print(response)

 The European Project PATAFEST consists of six work packages (WP):

1. WP1: Project Management - led by FUNDACION KINDE ACEBO (FUND)
2. WP2: Potato Health and Quality - spearheaded by Universitat Autonoma de Barcelona (UAB)
3. WP3: Plant Health and Integrated Pest Management - coordinated by Agricultural University Plovdiv (AU-Plovdiv)
4. WP4: Value Chains and Business Models - managed by Knowledge Network for Innovation in Agro-food Sectors (KNEIA)
5. WP5: Dissemination, Communication and Exploitation - headed by European Union Plant Breeding Company (EUPAT)
6. WP6: Ethics and Societal Impact - overseen by Aarhus University (AU)

These work packages cover various aspects of the project, including project management, potato health and quality, plant health and integrated pest management, value chains and business models, dissemination, communication and exploitation, and ethics and societal impact.


In [12]:
response = query_engine.query("what participant or participants provide images of potatoes and potato plants for the PATAFEST european project?")
print(response)

 The Centre Régional Universitaire de Nantes (CRUN) is one of the partners in the PATAFEST European project that contributes by providing images of potatoes and potato plants for the project's objectives. Their expertise lies in identifying and detecting potato spoilage VOC biomarkers under postharvest cold storage conditions.


In [14]:
response = query_engine.query("list all objectives, tasks and deliverables divided by work package of the PATAFEST european project?")
print(response)

ReadTimeout: timed out

In [ ]:
response = query_engine.query("list all objectives, tasks and deliverables for every work package of the PATAFEST european project")
print(response)

In [16]:
query_engine = index.as_chat_engine(similarity_top_k=10)
response = query_engine.chat("what are the participants of the PATAFEST european project?")
print(response)

print("-"*50)

response = query_engine.chat("Oh interesting, tell me more.")
print(response)

The PATAFEST European project has eighteen participants. Thirty-three percent are small and medium-sized enterprises (SMEs), with one-third being large companies (11%). There are four research technology organizations (RTOs) (22%), four universities (22%), and two associations (17%). Notable participants include FUNDITEC from Spain as the scientific/technical coordinator, NEIKER also from Spain as the scientific coordinator, the European Potato Trade Association (EUPAT), ECSA as associations, and INIAP from Ecuador. They aim to develop sustainable IPM solutions to address Zebra Chip during potato cultivation and soil-borne pathogens during postharvest storage.
--------------------------------------------------
 The PATAFEST European project is focused on addressing the challenges of Zebra Chip disease in potatoes during cultivation and soil-borne pathogens during postharvest storage through sustainable Integrated Pest Management (IPM) solutions. This collaborative initiative brings tog